In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.05)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.55 sec - Loss 0.103323 - ACC 82.73% - ACC Mean 82.73% - AUC 86.74% - AUC Mean 86.74% - Deter 000
Ite 00050 - 0.27 sec - Loss 0.096753 - ACC 81.76% - ACC Mean 82.17% - AUC 86.62% - AUC Mean 86.92% - Deter 016
Ite 00100 - 0.53 sec - Loss 0.079836 - ACC 80.60% - ACC Mean 81.53% - AUC 85.68% - AUC Mean 86.54% - Deter 066
Ite 00150 - 0.52 sec - Loss 0.051980 - ACC 80.23% - ACC Mean 81.66% - AUC 86.00% - AUC Mean 86.20% - Deter 116
Ite 00200 - 0.30 sec - Loss 0.035170 - ACC 79.86% - ACC Mean 80.05% - AUC 85.71% - AUC Mean 85.79% - Deter 166
Ite 00250 - 0.45 sec - Loss 0.030423 - ACC 79.86% - ACC Mean 80.24% - AUC 85.70% - AUC Mean 85.92% - Deter 216
Ite 00300 - 0.38 sec - Loss 0.028985 - ACC 80.19% - ACC Mean 80.57% - AUC 85.83% - AUC Mean 86.26% - Deter 266
Ite 00350 - 0.10 sec - Loss 0.028292 - ACC 81.30% - ACC Mean 80.61% - AUC 86.25% - AUC Mean 86.21% - Deter 316
Ite 00400 - 0.10 sec - Loss 0.027777 - ACC 80.53% - ACC Mean 80.62% - AUC 87.32% - AUC Mean 86.25% - Deter 366
I

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 80.86% +- 0.0% - auc: 88.65% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 84.98% +- 2.5% - auc: 92.0% +- 2.74%
SFIL RUN 2/10 - acc: 81.73% +- 4.24% - auc: 87.84% +- 4.1%
SFIL RUN 3/10 - acc: 81.53% +- 5.78% - auc: 89.08% +- 4.22%
SFIL RUN 4/10 - acc: 83.23% +- 4.06% - auc: 88.93% +- 4.66%
SFIL RUN 5/10 - acc: 81.53% +- 1.69% - auc: 88.02% +- 2.14%
SFIL RUN 6/10 - acc: 80.9% +- 5.21% - auc: 87.99% +- 3.42%
SFIL RUN 7/10 - acc: 82.45% +- 4.46% - auc: 88.92% +- 5.51%
SFIL RUN 8/10 - acc: 82.73% +- 2.97% - auc: 89.24% +- 1.87%
SFIL RUN 9/10 - acc: 81.93% +- 5.49% - auc: 87.99% +- 5.15%
SFIL RUN 10/10 - acc: 80.22% +- 4.5% - auc: 85.86% +- 4.09%
SFIL GLOBAL - acc: 82.12% +- 1.26% - auc: 88.59% +- 1.47%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 81.78% +- 4.02% - auc: 89.44% +- 4.03%
PFIL RUN 2/10 - acc: 80.89% +- 5.19% - auc: 88.29% +- 3.4%
PFIL RUN 3/10 - acc: 80.7% +- 2.35% - auc: 89.96% +- 1.29%
PFIL RUN 4/10 - acc: 82.68% +- 4.45% - auc: 91.29% +- 3.65%
PFIL RUN 5/10 - acc: 79.76% +- 7.35% - auc: 88.57% +- 3.96%
PFIL RUN 6/10 - acc: 80.29% +- 3.26% - auc: 90.93% +- 2.94%
PFIL RUN 7/10 - acc: 81.17% +- 3.36% - auc: 90.23% +- 3.48%
PFIL RUN 8/10 - acc: 84.57% +- 4.18% - auc: 93.63% +- 2.16%
PFIL RUN 9/10 - acc: 81.84% +- 5.78% - auc: 90.63% +- 3.23%
PFIL RUN 10/10 - acc: 80.69% +- 5.29% - auc: 89.35% +- 3.53%
PFIL GLOBAL - acc: 81.44% +- 1.31% - auc: 90.23% +- 1.46%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 81.28% +- 4.35% - auc: 85.97% +- 4.88%
SPOL RUN 2/10 - acc: 80.76% +- 4.12% - auc: 86.22% +- 5.82%
SPOL RUN 3/10 - acc: 82.29% +- 4.06% - auc: 89.27% +- 4.71%
SPOL RUN 4/10 - acc: 82.31% +- 3.61% - auc: 87.5% +- 4.23%
SPOL RUN 5/10 - acc: 84.51% +- 3.92% - auc: 90.94% +- 3.1%
SPOL RUN 6/10 - acc: 82.45% +- 4.11% - auc: 87.55% +- 3.63%
SPOL RUN 7/10 - acc: 83.02% +- 3.77% - auc: 89.94% +- 3.47%
SPOL RUN 8/10 - acc: 83.43% +- 1.64% - auc: 89.26% +- 2.35%
SPOL RUN 9/10 - acc: 83.79% +- 4.54% - auc: 88.55% +- 6.08%
SPOL RUN 10/10 - acc: 79.46% +- 4.05% - auc: 89.08% +- 4.4%
SPOL GLOBAL - acc: 82.33% +- 1.43% - auc: 88.43% +- 1.52%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 83.49% +- 2.41% - auc: 89.5% +- 3.93%
PPOL RUN 2/10 - acc: 80.1% +- 5.16% - auc: 88.03% +- 3.14%
PPOL RUN 3/10 - acc: 81.33% +- 3.52% - auc: 90.13% +- 3.73%
PPOL RUN 4/10 - acc: 83.34% +- 5.7% - auc: 91.46% +- 2.92%
PPOL RUN 5/10 - acc: 84.86% +- 4.73% - auc: 90.89% +- 3.96%
PPOL RUN 6/10 - acc: 81.96% +- 2.96% - auc: 92.37% +- 1.62%
PPOL RUN 7/10 - acc: 83.03% +- 6.57% - auc: 90.68% +- 3.86%
PPOL RUN 8/10 - acc: 83.26% +- 2.89% - auc: 91.84% +- 1.9%
PPOL RUN 9/10 - acc: 83.11% +- 2.18% - auc: 89.3% +- 2.72%
PPOL RUN 10/10 - acc: 82.32% +- 6.48% - auc: 90.07% +- 2.94%
PPOL GLOBAL - acc: 82.68% +- 1.25% - auc: 90.43% +- 1.23%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is even with MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is even with SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is even with PFIL
Metric ACC - OUR METHOD is even with SPOL
Metric AUC - OUR METHOD is even with SPOL
Metric ACC - OUR METHOD is even with PPOL
Metric AUC - OUR METHOD is even with PPOL
